In [24]:
import os

# Check these before running
# KPUM Workstation
studydir = ''
niftibasefolder = 'nifti' # nifti basefolder to go within studydir

# Finn's Laptop
studydir = '/mnt/s/Research/Projects/KPUM_NODDI/Data'
niftibasefolder = 'nifti' # nifti basefolder to go within studydir

# tsv-files to keep track of in studydir/niftibasefolder
subjecttrackerpath = os.path.join(studydir, niftibasefolder)
subjecttrackerfile = 'Subject_Tracker_for_sourcedata2nifti.tsv'

# Visualization script in code directory
visualizationscript = os.path.join(studydir,'code/kpum_noddi/shell','QC_visualize_nifti.sh')

###################################################################################
# USER INPUT

# Participant details
subject = '0XX' # e.g. 005
session = 'MR1' #i.e. 'MR1' or 'MR2'

In [ ]:
# Create conversion file sourcedata2nifti.tsv
import os, subprocess
import pandas as pd

# Define I/O folders and files
niftifolder = os.path.join(niftibasefolder, os.path.join(f'sub-{subject}',f'ses-{session}'))
niftioutputfolder = os.path.join(studydir, niftifolder)
subjecttrackertsv = os.path.join(subjecttrackerpath, subjecttrackerfile)

currdir = os.getcwd()
os.chdir(studydir)

# Read the subjecttracker file
df = pd.read_csv(subjecttrackertsv, sep="\t")
df.fillna('', inplace=True)

# Check if ready for QC

if [ df.loc[df['output'] == niftifolder ]['QC'] == 'Pending' ] :
  print(f"Starting visual QC for {niftifolder}")
  # Launch script for Visual QC
  p=subprocess.Popen(f"bash {visualizationscript} {subject} {session}", stdout=subprocess.PIPE, shell=True)
  # Poll process.stdout to show stdout live
  while True:
    output = p.stdout.readline()
    if p.poll() is not None:
      break
    if output:
      print(output.strip().decode("utf-8"))
  rc = p.poll()

  QCstatusDone = 'y' #int(input("Finished editing and saved the 'session_QC.tsv' file [y/n]?"))
  if QCstatusDone == 'y':
    # We update in subjectrackerfile
    df.loc[df['output'] == niftifolder, ['QC']] = 'Done'
    # sort the rows according to 'input'
    df = df.sort_values(by = 'input')
    # and write to subjecttrackertsv-file
    df.to_csv(subjecttrackertsv, sep="\t", index=False)
elif [ df.loc[df['output'] == niftifolder ]['QC'] == 'Done' ] :
  print(f"Visual QC already performed on {niftifolder}")
else:
  print(f"Status not yet ready for visual QC on {niftifolder}")
      
print('End of script')
os.chdir(currdir)
